# Preprocessing

In [1]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

def add_input_target_pair(input_text, target_text):
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

lines = []    
with open('data/aspell.txt', 'r') as f:
    raw = [line.strip() for line in f.read().lower().replace('_', ' ').split('\n')]
    processed = []
    raw_target = ''
    for line in raw:
        if line.startswith('$'):
            raw_target = line.replace('$', '')
        elif line != raw_target:
            processed.append(line + '\t' + raw_target)
    lines += processed

with open('data/wikipedia-misspellings.txt', 'r') as f:
    lines += [line.strip().replace('->','\t') for line in f.read().lower().split('\n')]
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    # Also needed to split target on commas, since some misspellings
    # have multiple targets in the data
    for target in target_text.split(','):
        target = '\t' + target.strip() + '\n'
        add_input_target_pair(input_text, target)        

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) 
# containing a one-hot vectorization of the English sentences.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Using TensorFlow backend.


tf.estimator package not installed.
tf.estimator package not installed.
Number of samples: 5041
Number of unique input tokens: 29
Number of unique output tokens: 38
Max sequence length for inputs: 19
Max sequence length for outputs: 22


# Model definition, training

In [2]:
# Model construction and training

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 4032 samples, validate on 1009 samples
Epoch 1/100
4032/4032 [==============================] - 8s 2ms/step - loss: 1.2820 - val_loss: 1.2761
Epoch 2/100
4032/4032 [==============================] - 6s 1ms/step - loss: 1.1943 - val_loss: 1.2413
Epoch 3/100
4032/4032 [==============================] - 5s 1ms/step - loss: 1.1050 - val_loss: 1.1410
Epoch 4/100
4032/4032 [==============================] - 6s 1ms/step - loss: 1.0203 - val_loss: 1.1136
Epoch 5/100
4032/4032 [==============================] - 6s 1ms/step - loss: 0.9463 - val_loss: 1.0677
Epoch 6/100
4032/4032 [==============================] - 6s 1ms/step - loss: 0.8883 - val_loss: 1.0505
Epoch 7/100
4032/4032 [==============================] - 6s 1ms/step - loss: 0.8440 - val_loss: 1.0350
Epoch 8/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.8053 - val_loss: 1.0388
Epoch 9/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.7656 - val_loss: 1.0054
Epoch 10/100
4032/4032 [=

4032/4032 [==============================] - 5s 1ms/step - loss: 0.0362 - val_loss: 1.2571
Epoch 80/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0352 - val_loss: 1.2447
Epoch 81/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0346 - val_loss: 1.2282
Epoch 82/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0345 - val_loss: 1.2550
Epoch 83/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0333 - val_loss: 1.2718
Epoch 84/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0350 - val_loss: 1.2705
Epoch 85/100
4032/4032 [==============================] - 6s 1ms/step - loss: 0.0309 - val_loss: 1.2550
Epoch 86/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0313 - val_loss: 1.2545
Epoch 87/100
4032/4032 [==============================] - 5s 1ms/step - loss: 0.0315 - val_loss: 1.2663
Epoch 88/100
4032/4032 [==============================] - 5s 1ms/step - loss:

C:\tools\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


# Inference

In [3]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [15]:
test_words = ['wrod', 'testign', 'aqcuire', 'halp', 'greuling']
encoder_input_test_data = np.zeros(
    (len(test_words), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, word in enumerate(test_words):
    for t, char in enumerate(word):
        if char in input_token_index:
            encoder_input_test_data[i, t, input_token_index[char]] = 1.

for idx in range(len(encoder_input_test_data)):
    print(test_words[idx],'->', decode_sequence(encoder_input_test_data[idx:idx+1]))


wrod -> forward

testign -> testive

aqcuire -> acquited

halp -> help

greuling -> griding

